Hopefully, This script will allow me to load up and fix up the moored data

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw 
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import pickle
import cmocean
import json
import f90nml
from collections import OrderedDict
from matplotlib.colors import LogNorm

fs=16
mpl.rc('xtick', labelsize=fs)
mpl.rc('ytick', labelsize=fs)
mpl.rc('legend', fontsize=fs)
mpl.rc('axes', titlesize=fs)
mpl.rc('axes', labelsize=fs)
mpl.rc('figure', titlesize=fs)
mpl.rc('font', size=fs)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')

import warnings
#warnings.filterwarnings('ignore')
from IPython.display import Markdown, display

%matplotlib inline

In [22]:
saveloc='/ocean/kflanaga/MEOPAR/mooredData'
year=2015
Mooring='Dockton'

In [23]:
df = pd.read_csv(f'{saveloc}/{Mooring}_1_1_{year}_to_12_31_{year}.csv')

In [24]:
Lat=float(df['**Legend**'][20][24:32])
Lon=float(df['**Legend**'][20][39:49])

In [25]:
print(Lat)
print(Lon)

47.37611
-122.45722


In [26]:
df.drop(df.index[[tuple(range(0,48))]],inplace=True)

In [27]:
df.columns=df.iloc[0]

In [28]:
df.drop(df.index[[0]],inplace=True)
df.drop(df.index[[-1]],inplace=True)

In [29]:
df=df.reset_index()

In [30]:
df=df.drop('index',axis=1)
df=df.drop('Sonde_ID',axis=1)
df=df.drop('Sonde_Batt_V',axis=1)
df=df.drop('Logger_Batt_V',axis=1)

In [31]:
df.columns

Index(['Date', 'Depth_m', 'Qual_Depth', 'Water_Temperature_degC',
       'Qual_Water_Temperature', 'Salinity_PSU', 'Qual_Salinity',
       'Chlorophyll_Fluorescence_ug/L', 'Qual_Chlorophyll_Fluorescence'],
      dtype='object', name=48)

In [32]:
df[df.columns[1:-1]]=df[df.columns[1:-1]].apply(lambda col:pd.to_numeric(col, errors='coerce'))

In [33]:
df['Z']=df['Depth_m']
#df.head()

In [34]:
df['dtUTC']=pd.to_datetime(df['Date'],format='%m/%d/%Y %I:%M:%S %p')
df['Lat']=Lat
df['Lon']=Lon
df['Z']=df['Depth_m']
press=gsw.p_from_z(-1*df['Z'],df['Lat'])
df['SA']=gsw.SA_from_SP(df['Salinity_PSU'],press,
                       df['Lon'],df['Lat'])
df['CT']=gsw.CT_from_t(df['SA'],df['Water_Temperature_degC'],press)
df['Chl']=df['Chlorophyll_Fluorescence_ug/L']
df['YD']=et.datetimeToYD(df['dtUTC'])

In [35]:
dfg=df.groupby(by='YD')

In [36]:
df_daily_avg=dfg.mean()
df_daily_avg['Lat']=Lat
df_daily_avg['Lon']=Lon

In [37]:
UTC=[]
for yd in range(0,len(dfg)):
    start = dt.datetime(year,1,1,12,0,0)      
    delta = dt.timedelta(yd)     
    offset = start + delta 
    UTC.append(offset)

In [38]:
df_daily_avg
df_daily_avg['dtUTC']=UTC

In [39]:
##### Saving data as Pickle files to be used in the summary file
saveloc='/ocean/kflanaga/MEOPAR/mooredData'
with open(os.path.join(saveloc,f'data_{Mooring}_{year}.pkl'),'wb') as hh:
    pickle.dump(df,hh)
    
with open(os.path.join(saveloc,f'daily_data_{Mooring}_{year}.pkl'),'wb') as hh:
    pickle.dump(df_daily_avg,hh)